<a href="https://colab.research.google.com/github/cokoroafor11/Playlist-Recommender-App/blob/main/flaskApp/recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building a Spotify Recommendation System

## Imports and Credentials

In [1]:
!pip install spotipy --upgrade
!pip install gdown
!pip install vaderSentiment
!pip install swifter
!pip install pymysql
!pip install sqlalchemy

import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from ast import literal_eval
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from google.colab import files
import swifter
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler


import pymysql
from sqlalchemy import create_engine, text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=392e13ed96f40c65bdcbe3d8b50649914701830935aeac527ecaaf057b82375f
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.3 MB/s eta 0:00:00


In [2]:
#Save client credentials
client_id = "c1f74565be774e65aa211462aaf5fed8"
client_secret = "2edce4052f8f46639c0e112658572d66"
#Create object
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,client_secret=client_secret),requests_timeout=100,retries=3)

In [3]:
#DB Instance Identifier:
username = 'admin'
pw = 'NeduMusic2023!!'
hostname = 'music-recommender-db.cwy9ickxbsdg.us-east-2.rds.amazonaws.com'
db_name = 'featureDB'
port = 3306

## Feature Database Import

In [4]:
def excel_list_to_df(file):
    '''Create the dataframe from the excel sheet of playlists'''

    df = pd.read_excel(file)
    #playlists = df['Link']
    return df

In [5]:
#!gdown 'https://docs.google.com/uc?id=1xq_oFsO6s1OgiRYFzyTv2lXy0pghUJ0Y&export=download'
#features_df = excel_list_to_df('/content/spotify_features.xlsx')

In [6]:
#Open connection
connection = pymysql.connect(host= hostname, user= username,password= pw)
cursor = connection.cursor()
#create engine
engine = create_engine('mysql+pymysql://' + username + ':' + pw + '@' + hostname + ':' + str(port) + '/' + db_name , echo=False)
conn = engine.connect()
query = text('''SELECT * FROM features''')
features_df =pd.read_sql(query,conn)

#Close connection
cursor.close()
connection.close()


In [7]:
features_df

,track_name,artists,song_popularity,artist_popularity,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_link,playlist_link
0,(It Goes Like) Nanana - Edit,['Peggy Gou'],94,74,"['electra', 'float house']",0.6710,0.8760,7,-5.681,0,0.0352,0.12000,0.188000,0.0823,0.9640,129.998,spotify:track:23RoR84KodL5HWvUTneQ1w,37i9dQZF1DXcBWIGoYBM5M
1,Barbie World (with Aqua) [From Barbie The Album],"['Nicki Minaj', 'Ice Spice', 'Aqua']",95,86,"['hip pop', 'pop', 'queens hip hop', 'rap']",0.7700,0.5800,0,-8.393,1,0.2470,0.51900,0.000127,0.2330,0.7530,144.072,spotify:track:741UUVE2kuITl0c6zuqqbO,37i9dQZF1DXcBWIGoYBM5M
2,Paint The Town Red,['Doja Cat'],92,84,"['dance pop', 'pop']",0.8680,0.5380,5,-8.603,1,0.1740,0.26900,0.000003,0.0901,0.7320,99.968,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,37i9dQZF1DXcBWIGoYBM5M
3,Seven (feat. Latto) (Explicit Ver.),"['Jung Kook', 'Latto']",100,85,['k-pop'],0.8020,0.8320,11,-4.107,1,0.0434,0.31100,0.000000,0.0815,0.8900,124.997,spotify:track:7x9aauaA9cu6tyfpHnqDLo,37i9dQZF1DXcBWIGoYBM5M
4,bad idea right?,['Olivia Rodrigo'],90,85,['pop'],0.6280,0.8780,9,-3.468,1,0.0901,0.00196,0.000012,0.0649,0.7960,129.976,spotify:track:2i8f4VnnBjy0yDqH2C452a,37i9dQZF1DXcBWIGoYBM5M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,"Young Birches, Op. 128, No. 2","['Amy Beach', 'Joanne Polk']",52,42,"['american romanticism', 'post-romantic era']",0.2400,0.0834,6,-29.545,0,0.0499,0.97800,0.944000,0.0804,0.1880,169.621,spotify:track:4VVEfBXep7FdOt524jkfLF,37i9dQZF1DWWEJlAGA9gs0
4031,"Violin Concerto In D Major, Op. 35: III. Final...","['Erich Wolfgang Korngold', 'Anne-Sophie Mutte...",42,34,"['classic soundtrack', 'early modern classical']",0.4050,0.3180,9,-17.140,1,0.0565,0.93800,0.723000,0.2840,0.3420,151.434,spotify:track:7hOLwSaTSkrqxSMzj4bXI1,37i9dQZF1DWWEJlAGA9gs0
4032,"Tahiti Trot, Op. 16 (orchestral transcription ...","['Dmitri Shostakovich', 'Russian State Symphon...",46,56,"['classical', 'compositional ambient', 'russia...",0.2910,0.0981,8,-23.156,1,0.0398,0.95000,0.865000,0.2750,0.1940,138.168,spotify:track:76i4IBw3KsuDw783tvHB0e,37i9dQZF1DWWEJlAGA9gs0
4033,The Promise,['Michael Nyman'],53,50,"['british contemporary classical', 'british so...",0.0992,0.2260,9,-14.840,0,0.0339,0.93700,0.903000,0.1830,0.0605,79.518,spotify:track:53ann6bENp7BF8uklrzQAp,37i9dQZF1DWWEJlAGA9gs0


## Functions for Creating Feature Set

In [8]:
def extract_songs(playlist):
    #Make sure playlist input is string
    if type(playlist) != str:
        playlist = str(playlist)

    results = spotify.playlist_items(playlist)
    tracks = results['items']
    uris = []
    while results['next']:
        results = spotify.next(results)
        tracks.extend(results['items'])

    #Append tracks to list if no type error
    for elem in tracks:
        try:
            uris.append(elem['track']['uri'])
        except TypeError:
            continue
    return uris

def get_artist_info(track):
    '''Helper function for get_track_info function'''
    artist_info = {}
    artist_id = track["artists"][0]["id"]
    artist = spotify.artist(artist_id)
    artist_pop = artist["popularity"]
    artist_genres = artist["genres"]

    artist_info['artist_genres'] = artist_genres
    artist_info['artist_popularity'] = artist_pop
    return artist_info

def get_track_info(track):
    track = spotify.track(track)
    track_info = {}
    artists= []
    artist_info = get_artist_info(track)
    track_info['name'] = track['name']
    for artist in track['artists']:
        artists.append(artist['name'])
    track_info['artists'] = artists
    track_info['popularity'] = track['popularity']
    return track_info, artist_info

def populate_song_info(playlist):
    '''
    Function that populates an dictionary where keys are the audio features and values are an array of scores for each song
    This function performs the lionshare of data setup
    '''
    songlist = extract_songs(playlist)

    #Storage for various types of data
    song_data = {}
    name_array = []
    artist_array = []
    song_pop_array = []
    artist_pop_array = []
    genres_array = []
    dance_array = []
    energy_array = []
    key_array = []
    loudness_array = []
    mode_array = []
    speech_array = []
    acoustic_array = []
    instrument_array = []
    live_array = []
    valence_array = []
    tempo_array = []
    song_link_array = []

    #Loop to append song info to appropriate array
    for song in songlist:

        features = spotify.audio_features(song)[0]
        #Check if features don't exist, skip the song entry
        if features == None:
            continue

        #Get song names, artists, popularity, and genres
        track, artist = get_track_info(song)
        name_array.append(track['name'])
        artist_array.append(track['artists'])
        song_pop_array.append(track['popularity'])
        artist_pop_array.append(artist['artist_popularity'])
        genres_array.append(artist['artist_genres'])

        #Append feature to each corresponding array
        dance_array.append(features['danceability'])
        energy_array.append(features['energy'])
        key_array.append(features['key'])
        loudness_array.append(features['loudness'])
        mode_array.append(features['mode'])
        speech_array.append(features['speechiness'])
        acoustic_array.append(features['acousticness'])
        instrument_array.append(features['instrumentalness'])
        live_array.append(features['liveness'])
        valence_array.append(features['valence'])
        tempo_array.append(features['tempo'])
        song_link_array.append(song)

    #Put all song data in a library with proper labels
    song_data['track_name'] = name_array
    song_data['artists'] = artist_array
    song_data['song_popularity'] = song_pop_array
    song_data['artist_popularity'] = artist_pop_array
    song_data['genres'] = genres_array
    song_data['danceability'] = dance_array
    song_data['energy'] = energy_array
    song_data['key'] = key_array
    song_data['loudness'] = loudness_array
    song_data['mode'] = mode_array
    song_data['speechiness'] = speech_array
    song_data['acousticness'] = acoustic_array
    song_data['instrumentalness'] = instrument_array
    song_data['liveness'] = live_array
    song_data['valence'] = valence_array
    song_data['tempo'] = tempo_array
    song_data['song_link'] = song_link_array

    song_db = pd.DataFrame(song_data)
    song_db['playlist_link'] = playlist
    return song_db

## Data Pre-Processing

In [9]:
features_df.dropna(inplace=True)
features_df

,track_name,artists,song_popularity,artist_popularity,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song_link,playlist_link
0,(It Goes Like) Nanana - Edit,['Peggy Gou'],94,74,"['electra', 'float house']",0.6710,0.8760,7,-5.681,0,0.0352,0.12000,0.188000,0.0823,0.9640,129.998,spotify:track:23RoR84KodL5HWvUTneQ1w,37i9dQZF1DXcBWIGoYBM5M
1,Barbie World (with Aqua) [From Barbie The Album],"['Nicki Minaj', 'Ice Spice', 'Aqua']",95,86,"['hip pop', 'pop', 'queens hip hop', 'rap']",0.7700,0.5800,0,-8.393,1,0.2470,0.51900,0.000127,0.2330,0.7530,144.072,spotify:track:741UUVE2kuITl0c6zuqqbO,37i9dQZF1DXcBWIGoYBM5M
2,Paint The Town Red,['Doja Cat'],92,84,"['dance pop', 'pop']",0.8680,0.5380,5,-8.603,1,0.1740,0.26900,0.000003,0.0901,0.7320,99.968,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,37i9dQZF1DXcBWIGoYBM5M
3,Seven (feat. Latto) (Explicit Ver.),"['Jung Kook', 'Latto']",100,85,['k-pop'],0.8020,0.8320,11,-4.107,1,0.0434,0.31100,0.000000,0.0815,0.8900,124.997,spotify:track:7x9aauaA9cu6tyfpHnqDLo,37i9dQZF1DXcBWIGoYBM5M
4,bad idea right?,['Olivia Rodrigo'],90,85,['pop'],0.6280,0.8780,9,-3.468,1,0.0901,0.00196,0.000012,0.0649,0.7960,129.976,spotify:track:2i8f4VnnBjy0yDqH2C452a,37i9dQZF1DXcBWIGoYBM5M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,"Young Birches, Op. 128, No. 2","['Amy Beach', 'Joanne Polk']",52,42,"['american romanticism', 'post-romantic era']",0.2400,0.0834,6,-29.545,0,0.0499,0.97800,0.944000,0.0804,0.1880,169.621,spotify:track:4VVEfBXep7FdOt524jkfLF,37i9dQZF1DWWEJlAGA9gs0
4031,"Violin Concerto In D Major, Op. 35: III. Final...","['Erich Wolfgang Korngold', 'Anne-Sophie Mutte...",42,34,"['classic soundtrack', 'early modern classical']",0.4050,0.3180,9,-17.140,1,0.0565,0.93800,0.723000,0.2840,0.3420,151.434,spotify:track:7hOLwSaTSkrqxSMzj4bXI1,37i9dQZF1DWWEJlAGA9gs0
4032,"Tahiti Trot, Op. 16 (orchestral transcription ...","['Dmitri Shostakovich', 'Russian State Symphon...",46,56,"['classical', 'compositional ambient', 'russia...",0.2910,0.0981,8,-23.156,1,0.0398,0.95000,0.865000,0.2750,0.1940,138.168,spotify:track:76i4IBw3KsuDw783tvHB0e,37i9dQZF1DWWEJlAGA9gs0
4033,The Promise,['Michael Nyman'],53,50,"['british contemporary classical', 'british so...",0.0992,0.2260,9,-14.840,0,0.0339,0.93700,0.903000,0.1830,0.0605,79.518,spotify:track:53ann6bENp7BF8uklrzQAp,37i9dQZF1DWWEJlAGA9gs0


In [10]:
features_df.dtypes

track_name            object
artists               object
song_popularity        int64
artist_popularity      int64
genres                object
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
song_link             object
playlist_link         object
dtype: object

In [11]:
features_df.iloc[0]['genres'][0]

'['

In [12]:
features_df['genres'] = features_df['genres'].apply(lambda row: literal_eval(row))

In [13]:
features_df.iloc[0]['genres'][0]

'electra'

In [14]:
features_df.iloc[0]['artists'][0]

'['

In [15]:
features_df['artists'] = features_df['artists'].apply(lambda row: literal_eval(row))

In [16]:
features_df.iloc[0]['artists'][0]

'Peggy Gou'

## Feature Engineering

### Sentiment Analysis

In [17]:
def sent_analysis(song_title):
  '''
  The sentiment analysis outputs positive, negative, neutral scores
  Combines into compound score that is a combo of all the above
  We will use the compound score, then put it into one of three buckets
  '''
  analyzer = SentimentIntensityAnalyzer()
  analysis = analyzer.polarity_scores(song_title)['compound']
  if analysis < -0.5:
    return 'negative'
  elif analysis > 0.5:
    return 'positive'
  else:
    return 'neutral'

In [18]:
sent_analysis(features_df['track_name'][0])

'neutral'

In [19]:
features_df['sentiment'] = features_df['track_name'].apply(lambda row: sent_analysis(row))

In [20]:
features_df['sentiment']

0        neutral
1        neutral
2        neutral
3        neutral
4       negative
          ...   
4030     neutral
4031     neutral
4032     neutral
4033     neutral
4034     neutral
Name: sentiment, Length: 4035, dtype: object

In [21]:
features_df['sentiment'].describe()

count        4035
unique          3
top       neutral
freq         3671
Name: sentiment, dtype: object

### Normalization

#### Float Columns

In [22]:
float_cols = features_df.select_dtypes(include=['float64'])
scaler = MinMaxScaler()
float_df = pd.DataFrame(scaler.fit_transform(float_cols), columns = float_cols.columns)
#Insert scaled columns into df
features_df= features_df.assign(**dict(float_df.items()))

#### Popularity Columns

In [23]:
pop_cols = features_df[['song_popularity','artist_popularity']]
scaler = MinMaxScaler()
pop_df = pd.DataFrame(scaler.fit_transform(pop_cols), columns = pop_cols.columns)
#Insert scaled columns into df
features_df= features_df.assign(**dict(pop_df.items()))

### Key Column

In [24]:
key = features_df[['key']]
scaler = MinMaxScaler()
key_df = pd.DataFrame(scaler.fit_transform(key), columns = key.columns)
#Insert scaled columns into df
features_df= features_df.assign(**dict(key_df.items()))

### One Hot Encoding

In [25]:
def one_hot(df, column):
  if type(column) != str: column = str(column)
  df = pd.get_dummies(df, columns=[column])
  return df

features_df = one_hot(features_df, 'sentiment')

### TF-IDF for Genres

In [26]:
#tfidf genre lists
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(features_df['genres'].apply(lambda x: " ".join(x)))
genres_df = pd.DataFrame(tfidf_matrix.toarray())
genres_df.columns = ['genre' + "_" + i for i in tfidf.get_feature_names_out()]
genres_df.reset_index(drop = True, inplace=True)

In [27]:
genres_df

,genre_150,genre_528hz,genre_5th,genre_abstract,genre_acid,genre_acoustic,genre_adult,genre_advocacy,genre_aesthetic,genre_african,...,genre_women,genre_wonky,genre_world,genre_worship,genre_yacht,genre_ye,genre_yodeling,genre_york,genre_youth,genre_zolo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
features_df = pd.concat([features_df,genres_df],axis = 1)
features_df.drop(columns = ['genres'], inplace = True)

In [29]:
features_df

,track_name,artists,song_popularity,artist_popularity,danceability,energy,key,loudness,mode,speechiness,...,genre_women,genre_wonky,genre_world,genre_worship,genre_yacht,genre_ye,genre_yodeling,genre_york,genre_youth,genre_zolo
0,(It Goes Like) Nanana - Edit,[Peggy Gou],0.94,0.6750,0.687500,0.877741,0.636364,0.860297,0,0.044110,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Barbie World (with Aqua) [From Barbie The Album],"[Nicki Minaj, Ice Spice, Aqua]",0.95,0.8250,0.788934,0.581114,0.000000,0.806164,1,0.309524,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Paint The Town Red,[Doja Cat],0.92,0.8000,0.889344,0.539025,0.454545,0.801972,1,0.218045,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Seven (feat. Latto) (Explicit Ver.),"[Jung Kook, Latto]",1.00,0.8125,0.821721,0.833648,1.000000,0.891714,1,0.054386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,bad idea right?,[Olivia Rodrigo],0.90,0.8125,0.643443,0.879746,0.818182,0.904469,1,0.112907,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,"Young Birches, Op. 128, No. 2","[Amy Beach, Joanne Polk]",0.52,0.2750,0.245902,0.083461,0.545455,0.383960,0,0.062531,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4031,"Violin Concerto In D Major, Op. 35: III. Final...","[Erich Wolfgang Korngold, Anne-Sophie Mutter, ...",0.42,0.1750,0.414959,0.318558,0.818182,0.631569,1,0.070802,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4032,"Tahiti Trot, Op. 16 (orchestral transcription ...","[Dmitri Shostakovich, Russian State Symphony O...",0.46,0.4500,0.298156,0.098192,0.727273,0.511487,1,0.049875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4033,The Promise,[Michael Nyman],0.53,0.3750,0.101639,0.226363,0.818182,0.677479,0,0.042481,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
features_df['song_link']

0       spotify:track:23RoR84KodL5HWvUTneQ1w
1       spotify:track:741UUVE2kuITl0c6zuqqbO
2       spotify:track:2IGMVunIBsBLtEQyoI1Mu7
3       spotify:track:7x9aauaA9cu6tyfpHnqDLo
4       spotify:track:2i8f4VnnBjy0yDqH2C452a
                        ...                 
4030    spotify:track:4VVEfBXep7FdOt524jkfLF
4031    spotify:track:7hOLwSaTSkrqxSMzj4bXI1
4032    spotify:track:76i4IBw3KsuDw783tvHB0e
4033    spotify:track:53ann6bENp7BF8uklrzQAp
4034    spotify:track:0qFZYO72iBOhW7X6KbL44Q
Name: song_link, Length: 4035, dtype: object

## User Playlist Feature Generation

In [31]:
#process to go from playlist to feature vector
#populate_song_info()
#sent_analysis
#one hot encode sent analysis

In [46]:
def gen_user_playlist_features(playlist):
  #Get basic feature df for users
  user_df = populate_song_info(playlist)

  #Sentiment Analysis
  user_df['sentiment'] = user_df['track_name'].apply(lambda row: sent_analysis(row))

  #Normalization
  ##Float columns
  #Get columns to scale
  float_cols = user_df.select_dtypes(include=['float64'])
  scaler = MinMaxScaler()
  float_df = pd.DataFrame(scaler.fit_transform(float_cols), columns = float_cols.columns)
  #Insert scaled columns into df
  user_df= user_df.assign(**dict(float_df.items()))
  ##Key and Popularity Columns
  #Get columns to scale
  pop_key_cols = user_df[['song_popularity','artist_popularity','key']]
  scaler = MinMaxScaler()
  pop_key_df = pd.DataFrame(scaler.fit_transform(pop_key_cols), columns = pop_key_cols.columns)
  #Insert scaled columns into df
  user_df= user_df.assign(**dict(pop_key_df.items()))

  #One hot encoding
  user_df = one_hot(user_df, 'sentiment')

  #Tfidf genre lists
  tfidf = TfidfVectorizer()
  tfidf_matrix =  tfidf.fit_transform(user_df['genres'].apply(lambda x: " ".join(x)))
  genres_df = pd.DataFrame(tfidf_matrix.toarray())
  genres_df.columns = ['genre' + "_" + i for i in tfidf.get_feature_names_out()]
  genres_df.reset_index(drop = True, inplace=True)
  user_df.drop(columns = ['genres'], inplace = True)
  user_df = pd.concat([user_df,genres_df],axis = 1)
  user_df.dropna(inplace=True)
  return user_df


In [47]:
def get_db_features():
  #Get info from sql
  #Open connection
  connection = pymysql.connect(host= hostname, user= username,password= pw)
  cursor = connection.cursor()
  #create engine
  engine = create_engine('mysql+pymysql://' + username + ':' + pw + '@' + hostname + ':' + str(port) + '/' + db_name , echo=False)
  conn = engine.connect()
  query = text('''SELECT * FROM features''')
  features_df = pd.read_sql(query,conn)

  #Close connection
  cursor.close()
  connection.close()
  features_df.dropna(inplace=True)

  #Sentiment Analysis
  features_df['sentiment'] = features_df['track_name'].apply(lambda row: sent_analysis(row))

  #Normalization
  ##Float columns
  #Get columns to scale
  float_cols = features_df.select_dtypes(include=['float64'])
  scaler = MinMaxScaler()
  float_df = pd.DataFrame(scaler.fit_transform(float_cols), columns = float_cols.columns)
  #Insert scaled columns into df
  features_df= features_df.assign(**dict(float_df.items()))
  ##Key and Popularity Columns
  #Get columns to scale
  pop_key_cols = features_df[['song_popularity','artist_popularity','key']]
  scaler = MinMaxScaler()
  pop_key_df = pd.DataFrame(scaler.fit_transform(pop_key_cols), columns = pop_key_cols.columns)
  #Insert scaled columns into df
  features_df= features_df.assign(**dict(pop_key_df.items()))

  #One hot encoding
  features_df = one_hot(features_df, 'sentiment')

  #Tfidf genre lists
  tfidf = TfidfVectorizer()
  tfidf_matrix =  tfidf.fit_transform(features_df_df['genres'].apply(lambda x: " ".join(x)))
  genres_df = pd.DataFrame(tfidf_matrix.toarray())
  genres_df.columns = ['genre' + "_" + i for i in tfidf.get_feature_names_out()]
  genres_df.reset_index(drop = True, inplace=True)
  features_df.drop(columns = ['genres'], inplace = True)
  features_df = pd.concat([features_df,genres_df],axis = 1)
  features_df.dropna(inplace=True)
  return features_df


In [48]:
user_playlist = gen_user_playlist_features('https://open.spotify.com/playlist/5PJUOcZUy72vVk3OW54nX8?si=162672425675420f')

In [49]:
#Get features of songs that aren't in this current playlist
def get_non_playlist_features(features_df, user_playlist_df):
  df_all = features_df.merge(user_playlist_df['song_link'], on=['song_link'],suffixes=('', '_x'), how='left', indicator=True)
  no_playlist_df = df_all[df_all['_merge'] == 'left_only']
  no_playlist_df = no_playlist_df.drop('_merge', axis=1)
  no_playlist_df.drop(list(no_playlist_df.filter(regex = '_x')), axis = 1, inplace = True)
  return no_playlist_df


In [50]:
user_playlist['song_link']

0      spotify:track:2zoNNEAyPK2OGDfajardlY
1      spotify:track:13ceaASJtv2OAXvKb86rtN
2      spotify:track:2HTBwbtFouEbIYiMV28S0n
3      spotify:track:4AtZRwSR8BOTTQg5ihSggt
4      spotify:track:3WyH1i3LoTWx8l80sllu3q
                       ...                 
221    spotify:track:0c6SqvH32BMgbEFvpHc2gs
222    spotify:track:6QqF0MqjQVotftZIuQUbvf
223    spotify:track:7onHzHeOVE3M9p0tNKQbXc
224    spotify:track:3d2hM2ZpQumxhOWURiGvGv
225    spotify:track:6MDdceLYec4AxohmorE4vH
Name: song_link, Length: 226, dtype: object

In [51]:
no_playlist_df = get_non_playlist_features(features_df,user_playlist)

In [52]:
def generate_recommendations(user_playlist, no_playlist_df,num_recommendations):
  cols = user_playlist.columns.union(no_playlist_df.columns)
  user_playlist = user_playlist.reindex(cols, axis=1, fill_value=0)
  no_playlist_df = no_playlist_df.reindex(cols,axis=1,fill_value=0)
  user_playlist_vector = user_playlist.drop(columns = ['track_name','artists','song_link','playlist_link']).sum()
  no_playlist_df['similarity'] = cosine_similarity(no_playlist_df.drop(columns = ['track_name','artists','song_link','playlist_link']).values,user_playlist_vector.values.reshape(1, -1))[:,0]
  no_playlist_df.sort_values('similarity',ascending = False, inplace = True)
  no_playlist_df.reset_index()
  #return no_playlist_df.drop(columns = ['track_name','artists','link']).values,user_playlist_vector
  return no_playlist_df.head(num_recommendations)[['track_name','artists','song_link','similarity']]

In [53]:
generate_recommendations(user_playlist,no_playlist_df,20)

,track_name,artists,song_link,similarity
1533,Number 9 (feat. Lil Yachty),"[Miguel, Lil Yachty]",spotify:track:6cU5urANuL1UYaBg5DNre4,0.961499
2125,You're Welcome,[Dwayne Johnson],spotify:track:6U4VqEHy4n5VeiH4pQPL24,0.956517
1733,RITMO DE TREINO,"[YOUTHISENDING, MC PR, MC Pequeno Diamante]",spotify:track:2pIC8fTfWwGjbo31YyNtzT,0.955319
1672,Automotivo Bibi Fogosa,"[Bibi Babydoll, Dj Brunin XM]",spotify:track:4kRLRWnn5u9wO2wOKAAdEN,0.952451
3794,Automotivo Bibi Fogosa,"[Bibi Babydoll, Dj Brunin XM]",spotify:track:4kRLRWnn5u9wO2wOKAAdEN,0.952451
1646,Automotivo Bibi Fogosa,"[Bibi Babydoll, Dj Brunin XM]",spotify:track:4kRLRWnn5u9wO2wOKAAdEN,0.952451
3765,MONTAGEM - PR FUNK,"[S3BZS, Mc Gw, Mc Menor Do Alvorada]",spotify:track:6njJR3OIp3oA0Kc4aC9Tzf,0.951461
1668,MONTAGEM - PR FUNK,"[S3BZS, Mc Gw, Mc Menor Do Alvorada]",spotify:track:6njJR3OIp3oA0Kc4aC9Tzf,0.951461
408,MONTAGEM - PR FUNK,"[S3BZS, Mc Gw, Mc Menor Do Alvorada]",spotify:track:6njJR3OIp3oA0Kc4aC9Tzf,0.951461
1657,MONTAGEM - PR FUNK,"[S3BZS, Mc Gw, Mc Menor Do Alvorada]",spotify:track:6njJR3OIp3oA0Kc4aC9Tzf,0.951461
